#### 一. Hotspot架构
* JIT在java程序运行中, 针对底层操作系统, 生成高表现力的本地机器代码
* JVM在随着不断演进, 持续迭代java runtime, 而且有多线程的垃圾收集器  
* JVM的主要组建包括:  
    * classloader
    * 运行时数据区
    * 执行引擎   
    
<img src="img/jvm1.png" width="60%" height="60%" align="left"/>  

* JVM的3个关键组件:  
    * 堆
    * JIT编辑器
    * 垃圾回收器     
  关键组件在下图用紫色标出, 调试java程序性能时, 通常使用'堆大小'和'选择哪个垃圾回收器'为主.   
  虽然JIT也关系着程序性能, 但只能在下个版本的jvm中给出优化
  
  
 <img src="img/jvm2.png" width="60%" height="60%" align="left"/>  
 
 

#### 二. Performance Basics  
* 性能优化的两个目标:  
    * 响应能力 (responsiveness)  
    响应能力表示程序作出响应所消耗的时间,  
    对于关注响应能力的程序, 长的停顿时间是不能被接收的,  典型的例子包括: 
        * 桌面UI对时间的响应时间
        * WebSite返回页面的时间
        * 数据库查询返回数据的时间  
    
    * 吞吐量 (throughput)  
    吞吐量表示一段时间内完成的任务数量  
    对于关注吞吐量的程序, 长时间的停顿可以被接受, 因为这些程序只关注长期表现, 不考虑短期内的响应, 典型例子包括:  
        * 给定时间内完成的事务数量  
        * 1小时内, 批处理程序完成的job数
        * 1小时内, 数据库完成的query数量
        
* 什么是垃圾自动回收  
gc程序一直在监视着heap(堆内存), 并且把正在使用或被引用的对象, 和不再使用或不被引用的对象区分开来.   
一个正在使用或被引用的对象, 说明程序中存在指向该对象的指针; 而不再使用或不被引用的对象说明不存在指向该对象的指针  
所以, 对于不再使用或不被引用的对象占用的内存, 可以被回收. 下面展示这个步骤        

#### 三. 垃圾回收的步骤  
* Step 1: 标记
    * gc收集器在此步区分开正在使用, 和未被使用的内存片
    * 为了区分哪些对象在使用, 哪些没有在使用, 所有对象都要被scan(扫描)一次. 这种操作很消耗时间
        * 如下图, 正在被引用的对象用蓝色标出, 没有引用的对象用金色标出  
<img src='img/marking1.png' height='60%' width='60%' align='left'>    

* Step 2: Normal Deletion
    * 该步骤将删除未被引用的对象(金色的), 使得memory allocator(内存分配器)持有指向free space的指针,   
      下次分配对象时用这些free space空间来产生对象  
      
 <img src='img/deletion.png' width='60%' height='60%' align='left'> 

* Step 2a: Deletion with Compacting
    * 为了提高表现, 也可以在deletion后, compact(压缩)剩余的内存空间. 通过将存在引用的对象聚合在一起, 这将使得新内存的分配更容易更快速  
    
 <img src='img/deletion2a.png' width='60%' height='60%' align='left'>     

#### 四. 为什么分代收集
1. 早期jvm的做法
    * 早期的jvm, gc收集器必须标记并压缩内存中的所有对象. 这种操作并不高效, 原因是随着越来越多的对象被生成, gc的时间也越来越长
    * 经过经验性的总结, 发现对象多是短生命周期的, 时间越久, 对象的存活率就越低
    * 如下图, x轴表示随时间增长, 共分配了多少内存; y轴表示随时间增长, 存活的对象有多大  
    
<img src='img/earlyjvm.png' width='50%' height='50%' align='left'>

2. JVM的代
    * 如上经验性分析可用于提高gc的表现力. 对象根据存活能力被分成青年带,老年代,永久带
    * 青年代:
        * 新对象再次产生, 青年代的gc称为'minor gc' 青年带上的对象有很高的死亡率. 所有在minor gc后幸存的对象都会增长一个age, 直到最终进入老年代.   
        * 充满死亡对象的青年代, 在进行gc时通常会很快
        * 所有的'minor gc'都是'stop the world'事件
    * 老年代  
        * 老年代存储长期存活的对象. 一般来讲, 会给青年代中的对象设置一个阈值, 当对象的age到达阈值时, 该对象就会从青年代移动到老年代  
        * 最终, 老年代的对象也会被gc掉. 考年代的gc称作'major gc'
        * 老年代的gc通常很慢, 因为这会涉及到所有其关联的存活对象, 所以对于'高响应' (responsiveness)的应用, 应该保证老年代的垃圾集合最小化  
        * 老年代gc的速度, 和选择的老年代gc收集器有很大关系
    * 永久代  
        * 永久代包含JVM描述类和方法的meta信息
        * java se的类和方法存在这个代中
        * 当JVM不在使用某个类时, 就会引发gc, 永久代的gc会触发'full gc' - 所有代进行gc

#### 五. gc的一般流程
现在, 已经讲明jvm为什么要分代(避免scan整个内存), 是时候讲这些空间之间如何交互了.下图这展示了object allocation和ageing process的过程
* 1.新对象被分配在eden区, 2个survivor区在jvm启动时是空的  
* 2.当eden区空间满时, 触发minor gc: 
    * 存在引用的对象移动到第一个survivor空间(s0,其age从0变成1), 不存在引用的对象被从eden区删除 (因此minor gc后,eden区为空)
    * 如下图, 存在引用的对象蓝色标记, 不存在引用的对象黄色标记, 对象上的数字表示age  
    
<img src='img/pro1.png' width='60%' height='60%' align='left'>

* 3.下次minor gc, 同样的事情发生在eden区, 未引用的对象从eden区中删除, 但是存在引用的对象会被移动到第二个survivor空间(s1,其age从0变为1); 此外, 上次minor gc移动到s0的对象, 将被移动到s1, 且其age增加1. 一旦所有对象都移动到s1, s0和eden区都会变成空的, 但是请注意, survivor区的对象的age不一样    

<img src='img/pro2.png' width='60%' height='60%' align='left'>

* 4.再下次minor gc, 同样的处理流程触发, 但survivor空间切换, 存在引用的对象会被移动到s0, s1和eden区会被清空
* 5.如下图所示, 每次monor gc,当存活对象的age到达一个阈值后(图中的阈值为8), 幸存对象将从青年代被提升到老年代(被称为promotion)  

<img src='img/pro3.png' width='60%' height='60%' align='left'>

* 6.随着minor gc的不断发生, 越来越多的对象被提升(promotion)到老年代  
* 7.随着青年代不断的执行上述整个流程, 最终触发老年代的major gc. 老年代中, 未被引用的对象将被清除, 存在引用的对象将被压缩到老年代的一端(clean up and compact)  

<img src='img/pro4.png' width='60%' height='60%' align='left'>

#### 六.  高性能gc器 - 替换默认的serial gc器
* Parallel GC
    * Parallel GC是多线程的年轻带收集器. 一个n核心的主机上默认使用n个线程进行gc  
      gc线程数控制: `-XX:ParallelGCThreads=<desired number>`
    * 2核心以上的主机使用parallel gc才能有更好的结果
    * 使用场景:  
      Parallel collector也叫做高吞吐收集器, 适用于收集大内存并且允许暂停时间稍长的应用. 如批处理任务,数据库query等
    * `-XX:+UseParallelGC`  
      该参数在青年带上使用多线程收集器, 在老年代上使用单线程收集器
    * `-XX:+UseParallelOldGC`  
      该参数在年轻带和年老带都使用多线程收集器,且在年老带使用压缩(compact)   
      HotSpot虚拟机仅仅在年老带使用压缩清除(compact), 在年轻带使用复制清除(copy), 因此年轻带不需要压缩

* CMS GC  
    * CMS收集器, 全称"Concurrent Mark Sweep (CMS) Collector", 是一种运行在老年代的低暂停,高响应收集器. 通过让垃圾收集线程和应用线程并行的方式最小化暂停时间
    * 这种并发的高响应收集器, 不会把已存在的对象进行copy或compact, 因此会在内存中留下缝隙, 如果这种方式是一个问题的话就要分配更大的老年代  
    * CMS算法在年轻带使用和Parallel GC相同的收集算法  
    * CMS高响应收集器可用在桌面UI,WebSite, 数据库查询响应等场景  
    * 使用CMS: `-XX:+UseConcMarkSweepGC`  
      指定收集线程个数: `-XX:ParallelCMSThreads=<n>`

* G1收集器
    * 在java1.7中被设计出来, 旨在替代CMS收集器, 他同样是高响应,多线程,与应用并行的收集器

#### 七. 默认收集器  
* `java -XX:+PrintCommandLineFlags -version ` : 查看当前gc是哪种
* Default garbage collectors:
    * Java 7 - Parallel GC
    * Java 8 - Parallel GC
    * Java 9 - G1 GC
    * Java 10 - G1 GC
* [JVM启动参数](https://www.baeldung.com/jvm-parameters)